# Implementation and linear cryptanalysis of a simplified AES-like cipher
Laboratory session 1 of *Information Security*, AY 2024/25

To Do:
- precautions for input shape or type mismatches
- proper function documentation

In [51]:
# library imports
import numpy as np
import sympy as sp
from sympy import Matrix
from itertools import product, combinations
from collections import Counter
import ast


## Task 1
Using a programming language of your choice, implement the encryptor for a simplified
AES-like cipher.

Soooo let's do helper functions for
- subkey generation
- substitution S
- transposition T
- linear transformation L

In [52]:
def substitution(v):
    return (2*v) % 11

def transposition(y):
    # returns copy of y with 2nd half flipped
    # assuming y has an even length
    h = len(y)//2 # halfway point
    z = np.copy(y)
    z[h:]= z[h:][::-1]
    return z

A = np.array([[2,5],[1,7]])
def linear(z):
    W = z.reshape((2,4))
    w = (A @ W) % 11
    return w.flatten()

def subkeyGen(K):
    K = np.array(K)
    idx = np.array([
        [0, 2, 4, 6],
        [0, 1, 2, 3],
        [0, 3, 4, 7],
        [0, 3, 5, 6],
        [0, 2, 5, 7],
        [2, 3, 4, 5]
    ])
    return K[idx]

Put everything together in an encryptor function:

In [53]:
def encryptA(u, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    # first subkey sum
    v = (u + np.concatenate((subkeys[0], subkeys[0]))) % 11

    for i in range(4):
        # S: perform substitution
        y = substitution(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = (w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11

    # last iteration without linear step:
    # S: perform substitution
    y = substitution(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = (z + np.concatenate((subkeys[5], subkeys[5])) ) % 11

    return v

Test with given test message & key:

In [54]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptA(u,k)

array([4., 0., 0., 9., 7., 0., 0., 3.])

## Task 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field F = GF(p).

The transposition (flipping the 2nd half of the vector) is its own inverse here. For the linear transformation,


In [55]:
def substitutionInv(v):
    return 6*v % 11

A_inv = np.array([[2, 8],[6, 10]])
def linearInv(w):
    W = w.reshape((2,4))
    z = (A_inv @ W) % 11
    return z.flatten()

def decryptA(x, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)

    # subkey diff
    z = (x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11
    # T^-1: inverse transposition
    y = transposition(z)
    # S^-1: inv. substitution
    v = substitutionInv(y)

    for i in range(0, 4):
        # subkey diff
        w = (v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11
        # L^-1: inverse linear trafo
        z = linearInv(w)
        # T^-^: inverse transposition
        y = transposition(z)
        # S^-1: inv. substitution
        v = substitutionInv(y)

    # subkey diff
    u = (v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11

    return u

In [56]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptA(encryptA(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

True


# Task 3

In [57]:
def linear_matrices():
    A_l = []
    B = []

    zero_vector = [0] * 8

    for j in range(8):
        k = [0] * 8  # k influence
        k[j] = 1
        x = encryptA(zero_vector, k)
        A_l.append(x)

    for j in range(8):
        u = [0] * 8
        u[j] = 1
        x = encryptA(u, zero_vector)
        B.append(x)

    A_l = np.array(A_l).T
    B = np.array(B).T

    return A_l, B

A_l, B = linear_matrices()

print("A =")
print(A_l)

print("B =")
print(B)

u = np.random.randint(0, 11, 8)
k = np.random.randint(0, 11, 8)

x1 = encryptA(u, k)
x2 = (A_l @ k + B @ u) % 11

print("u:", u)
print("k:", k)
print("\nx1 = encryptA(u, k):")
print(x1)
print("\nx2 = A @ k + B @ u:")
print(x2.astype(int))

print("\n Matches" if np.all(x1 == x2) else "No matches")

A =
[[ 9  0  1  6  0  0  1 10]
 [ 0  8  6  2  2  9  0  0]
 [ 0  6  0  8  3 10  0  0]
 [ 6  0  0  8  0  1  6  6]
 [ 2  0  1 10  0  0  1  3]
 [ 0  1  8  4  9  6  0  0]
 [ 0 10  0  5  7  6  0  0]
 [ 3  0  0  1  0  1  4  8]]
B =
[[6 0 0 3 3 0 0 0]
 [0 6 3 0 0 3 0 0]
 [0 3 6 0 0 0 3 0]
 [3 0 0 6 0 0 0 3]
 [5 0 0 0 4 0 0 8]
 [0 5 0 0 0 4 8 0]
 [0 0 5 0 0 8 4 0]
 [0 0 0 5 8 0 0 4]]
u: [0 1 1 1 6 7 2 1]
k: [10  1 10  5  4 10  1  6]

x1 = encryptA(u, k):
[ 3  8  8  7 10  4  5  0]

x2 = A @ k + B @ u:
[ 3  8  8  7 10  4  5  0]

 Matches


# Task 4

Carry out linear cryptanalysis

In [58]:

p = 11


def inv_matrix_mod(A_numpy, p):
    A_sym = sp.Matrix(A_numpy.tolist())
    det = A_sym.det() % p
    if sp.gcd(det, p) != 1:
        raise ValueError("Matrix not invertible")
    A_inv_sym = A_sym.inv_mod(p)
    return np.array(A_inv_sym.tolist(), dtype=int)

A_l, B = linear_matrices()
Ainv2 = inv_matrix_mod(A_l, p)


with open("KPApairsJ_linear.txt", "r") as file:
    lines = file.readlines()

for line in lines:
    if line.strip() == "":
        continue

u_first, x_first = lines[0].split('\t')
u = np.array(ast.literal_eval(u_first))
x = np.array(ast.literal_eval(x_first))

Bu = (B @ u) % p
x_minus_Bu = (x - Bu) % p
k = (Ainv2 @ x_minus_Bu) % p


print("The recovered key is")
print(k.astype(int).tolist())


for idx, line in enumerate(lines):
    u_str, x_str = line.split('\t')
    u = np.array(ast.literal_eval(u_str))
    x = np.array(ast.literal_eval(x_str))

    x_check = (A_l @ k + B @ u) % p

    print(f"\nThe pair #{idx + 1}")
    print("u:", u.tolist())
    print("x initial   ", x.tolist())
    print("x computed:  ", x_check.astype(int).tolist())

    if list(x_check.astype(int)) == list(x):
        print("Matches :)")
    else:
        print("Doesn't match")


The recovered key is
[7, 2, 3, 2, 4, 6, 6, 0]

The pair #1
u: [7, 6, 1, 6, 5, 2, 2, 6]
x initial    [5, 2, 9, 10, 3, 6, 2, 4]
x computed:   [5, 2, 9, 10, 3, 6, 2, 4]
Matches :)

The pair #2
u: [4, 7, 8, 2, 8, 3, 6, 2]
x initial    [6, 10, 0, 9, 1, 3, 6, 3]
x computed:   [6, 10, 0, 9, 1, 3, 6, 3]
Matches :)

The pair #3
u: [4, 5, 7, 6, 4, 10, 5, 10]
x initial    [6, 5, 7, 2, 5, 2, 9, 1]
x computed:   [6, 5, 7, 2, 5, 2, 9, 1]
Matches :)

The pair #4
u: [5, 8, 9, 1, 1, 3, 4, 4]
x initial    [10, 8, 3, 1, 5, 3, 3, 5]
x computed:   [10, 8, 3, 1, 5, 3, 3, 5]
Matches :)

The pair #5
u: [4, 2, 0, 1, 5, 3, 3, 0]
x initial    [5, 0, 5, 8, 6, 9, 9, 10]
x computed:   [5, 0, 5, 8, 6, 9, 9, 10]
Matches :)


# Task 5

In [59]:
s_table = np.array([0, 2, 4, 8, 6, 10, 1, 3, 5, 7, 9])
s_table_inv = np.argsort(s_table)

def substitutionB(v):
    return s_table[v]

def substitutionInvB(v):
    return s_table_inv[v]

def encryptB(u, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    # first subkey sum
    v = ((u + np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    for i in range(4):
        # S: substitution
        y = substitutionB(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = ((w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11).astype(int)

    # last iteration without linear step:
    # S: substitution
    y = substitutionB(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = ((z + np.concatenate((subkeys[5], subkeys[5])) ) % 11).astype(int)

    return v

def decryptB(x, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)

    # subkey diff
    z = ((x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11).astype(int)
    # inv. T
    y = transposition(z)
    # inv. S
    v = substitutionInvB(y)

    for i in range(0, 4):
        # subkey diff
        w = ((v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11).astype(int)
        # inv. L
        z = linearInv(w)
        # inv. T
        y = transposition(z)
        # inv. S
        v = substitutionInvB(y)

    # subkey diff
    u = ((v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    return u

Test encryption with given message & key:

In [60]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptB(u,k)

array([9, 0, 0, 0, 5, 0, 0, 6])

Test decryption:

In [61]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptB(encryptB(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

True


# Task 6

In [69]:

p = 11


# Compute approximate key for one (u, x) pair
def kpa_approx_attack(M_A, M_B, C, u, x):
    rhs = (C @ x - M_B @ u) % p
    M_A_inv = inv_matrix_mod(M_A, p)
    return (M_A_inv @ rhs) % p

# Aggregate approximate keys using the mode for each coordinate
def aggregate_keys(guesses):
    num_coords = len(guesses[0])
    aggregated = []
    for i in range(num_coords):
        values = [int(guess[i]) for guess in guesses]
        mode_value, _ = Counter(values).most_common(1)[0]
        aggregated.append(mode_value)
    return np.array(aggregated)

# Return all neighbors of the key within a given Hamming distance
def hamming_neighbors(key, max_distance, p):
    key = np.array(key)
    n = len(key)
    neighbors = []
    for d in range(max_distance + 1):
        for positions in combinations(range(n), d):
            if d == 0:
                neighbors.append(key)
            else:
                choices = []
                for i in positions:
                    choices.append([v for v in range(p) if v != key[i]])
                for new_vals in product(*choices):
                    candidate = key.copy()
                    for idx, pos in enumerate(positions):
                        candidate[pos] = new_vals[idx]
                    neighbors.append(candidate)
    return neighbors

# Compute candidate accuracy as the average fraction of zeros in the check vector for all pairs
def candidate_accuracy(candidate, u_list, x_list, M_A, M_B, C, p):
    accuracies = []
    for u, x in zip(u_list, x_list):
        approx_vector = (M_A @ candidate + M_B @ u + C @ x) % p
        zero_count = np.count_nonzero(approx_vector == 0)
        frac = zero_count / len(approx_vector)
        accuracies.append(frac)
    return np.mean(accuracies)

# Exhaustively improve candidate by searching its neighborhood
def exhaustive_improve_candidate(candidate, u_list, x_list, M_A, M_B, C, p, max_radius):
    best_candidate = candidate.copy()
    best_acc = candidate_accuracy(best_candidate, u_list, x_list, M_A, M_B, C, p)
    print("Initial accuracy of aggregated key:", best_acc)
    neighbors = hamming_neighbors(candidate, max_radius, p)
    print(f"Searching among {len(neighbors)} candidates (Hamming distance ≤ {max_radius})...")
    for cand in neighbors:
        acc = candidate_accuracy(cand, u_list, x_list, M_A, M_B, C, p)
        if acc > best_acc:
            best_candidate = cand.copy()
            best_acc = acc
            print(f"Found candidate: {best_candidate} with accuracy {best_acc:.4f}")
    return best_candidate, best_acc

# Example KPA pairs
pairs = [
    ([10, 4, 3, 0, 2, 6, 9, 4], [5, 6, 1, 3, 0, 10, 0, 1]),
    ([10, 2, 2, 3, 5, 9, 2, 3], [6, 10, 9, 0, 2, 9, 0, 5]),
    ([0, 7, 0, 7, 4, 3, 1, 10], [7, 10, 7, 4, 4, 1, 8, 8]),
    ([8, 4, 0, 8, 5, 5, 6, 5], [7, 0, 9, 6, 2, 1, 10, 6]),
    ([9, 2, 0, 3, 7, 2, 2, 6], [9, 0, 5, 0, 10, 7, 4, 1]),
]
u_list = [np.array(u) for u, _ in pairs]
x_list = [np.array(x) for _, x in pairs]

# Get matrices
M_A, M_B = linear_matrices()

# Set matrix C = I
C = np.eye(8, dtype=int)

# Compute approximate keys for each pair
approx_keys = []
print("Approximate keys for each pair:")
for i, (u, x) in enumerate(zip(u_list, x_list)):
    k_approx = kpa_approx_attack(M_A, M_B, C, u, x)
    print(f"Pair {i+1}: k̂ ≈ {k_approx}")
    approx_keys.append(k_approx)

# Aggregate the approximate keys
aggregated_key = aggregate_keys(approx_keys)
print("\nAggregated approximate key:", aggregated_key)

# Search for a better candidate in the neighborhood
max_radius = 4
improved_key, improved_acc = exhaustive_improve_candidate(aggregated_key, u_list, x_list, M_A, M_B, C, p, max_radius)
print("\nBest candidate after improvement:", improved_key)
print("Achieved accuracy:", improved_acc)


Approximate keys for each pair:
Pair 1: k̂ ≈ [1 8 6 7 1 9 7 4]
Pair 2: k̂ ≈ [ 6 10  8  6  7  1  1  1]
Pair 3: k̂ ≈ [7 8 6 4 8 0 2 0]
Pair 4: k̂ ≈ [ 1  0  1  6 10  0  0  5]
Pair 5: k̂ ≈ [ 3 10 10  8  0 10  7  2]

Aggregated approximate key: [1 8 6 6 1 0 7 4]
Initial accuracy of aggregated key: 0.125
Searching among 758881 candidates (Hamming distance ≤ 4)...
Found candidate: [1 2 6 6 1 0 7 4] with accuracy 0.1750
Found candidate: [1 8 8 6 1 0 7 4] with accuracy 0.2000
Found candidate: [5 8 8 6 1 0 7 4] with accuracy 0.2250
Found candidate: [1 8 6 9 1 8 7 4] with accuracy 0.2500
Found candidate: [10  8  9  6  1  3  7  4] with accuracy 0.2750
Found candidate: [4 1 3 5 1 0 7 4] with accuracy 0.3000

Best candidate after improvement: [4 1 3 5 1 0 7 4]
Achieved accuracy: 0.3


## Task 7
implement the encryptor for a simplified AES-like cipher with the following parameters ..

In [64]:
def subkeyGenC(K):
    idx = np.array([
        [0, 1, 2, 3],
        [0, 1, 3, 2],
        [1, 2, 3, 0],
        [0, 3, 1, 2],
        [2, 3, 0, 1],
        [1, 3, 0, 2]
    ])
    return K[idx]

# modular multiplicative inverse table for GF(11)
# for v[j] = 0, we set v[j]^-1 to 0
inv_table = np.array([0, 1, 6, 4, 3, 9, 2, 8, 7, 5, 10])
def substitutionC(v):
    return 2*inv_table[v] % 11

def substitutionInvC(y):
    return inv_table[(6*y) % 11]

def encryptC(u, k):
    # generate subkeys from key k
    subkeys = subkeyGenC(k)
    # first subkey sum
    v = ((u + np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    for i in range(4):
        # S: substitution
        y = substitutionC(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = ((w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11).astype(int)

    # last iteration without linear step:
    # S: substitution
    y = substitutionC(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = ((z + np.concatenate((subkeys[5], subkeys[5])) ) % 11).astype(int)

    return v

def decryptC(x, k):
    # generate subkeys from key k
    subkeys = subkeyGenC(k)

    # subkey diff
    z = ((x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11).astype(int)
    # inv. T
    y = transposition(z)
    # inv. S
    v = substitutionInvC(y)

    for i in range(0, 4):
        # subkey diff
        w = ((v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11).astype(int)
        # inv. L
        z = linearInv(w)
        # inv. T
        y = transposition(z)
        # inv. S
        v = substitutionInvC(y)

    # subkey diff
    u = ((v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    return u

In [65]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptC(u,k)

array([5, 0, 3, 2, 5, 2, 1, 1])

In [66]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptC(encryptC(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

True


# Task 8

In [67]:
def key_gen():
    return np.random.randint(0, 11, 4)

def meet_in_the_middle(encrypt_fn, decrypt_fn, P_list, C_list, key_samples=80000):
    print("Generating random key candidates...")

    # Generate unique random keys for K1 and K2
    keys1 = np.unique([key_gen() for _ in range(key_samples)], axis=0).tolist()
    keys2 = np.unique([key_gen() for _ in range(key_samples)], axis=0).tolist()

    print(f"Keys1: {len(keys1)}, Keys2: {len(keys2)}")

    print("\n→ Computing encryptions of all plaintexts with each k1...")
    enc_map = {}
    for i, k1 in enumerate(keys1):
        all_outputs = []
        for P in P_list:
            out = encrypt_fn(np.array(P), np.pad(k1, (0, 4), constant_values=0))
            all_outputs.append(tuple(out.tolist()))
        enc_map[tuple(all_outputs)] = k1

    print("\n← Computing decryptions of all ciphertexts with each k2 and matching...")
    for j, k2 in enumerate(keys2):
        all_outputs = []
        for C in C_list:
            out = decrypt_fn(np.array(C), np.pad(k2, (0, 4), constant_values=0))
            all_outputs.append(tuple(out.tolist()))

        key = tuple(all_outputs)
        if key in enc_map:
            k1 = enc_map[key]
            print("\nFound matching key pair!")
            print("k1:", k1)
            print("k2:", k2)
            return k1, k2

    print("No match found.")
    return None, None

In [68]:
plaintexts = [
    [4, 1, 6, 10, 2, 3, 5, 10],
    [10, 5, 4, 4, 7, 3, 2, 0],
    [2, 6, 8, 0, 6, 8, 10, 9],
    [3, 7, 2, 10, 1, 6, 9, 0],
    [5, 1, 6, 3, 10, 8, 8, 10],
]

ciphertexts = [
    [2, 1, 4, 0, 6, 7, 5, 5],
    [2, 8, 6, 2, 6, 2, 10, 0],
    [5, 5, 1, 4, 10, 2, 9, 2],
    [3, 8, 10, 10, 10, 9, 7, 8],
    [10, 0, 8, 10, 2, 10, 2, 2],
]

k1, k2 = meet_in_the_middle(encryptC, decryptC, plaintexts, ciphertexts, key_samples=80000)

Generating random key candidates...
Keys1: 14578, Keys2: 14573

→ Computing encryptions of all plaintexts with each k1...

← Computing decryptions of all ciphertexts with each k2 and matching...

Found matching key pair!
k1: [0, 3, 10, 10]
k2: [1, 9, 8, 0]
